In [1]:
import folium
from folium.features import GeoJsonTooltip
import fiona
import geopandas
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

import geopandas.tools
#from geopy.geocoders import Nominatim
#import geopy
from geopandas.tools import geocode
#from geopy.extra.rate_limiter import RateLimiter
from difflib import SequenceMatcher


import numpy as np

from shapely.geometry import Point, Polygon

In [2]:
# Import dataframes with station locations 

stations_2011 = pd.read_csv('cleaned_data/stations_2011.csv')
stations_2012 = pd.read_csv('cleaned_data/stations_2012.csv')
stations_2013 = pd.read_csv('cleaned_data/stations_2013.csv')
stations_2014 = pd.read_csv('cleaned_data/stations_2014.csv')
stations_2015 = pd.read_csv('cleaned_data/stations_2015.csv')
stations_2016 = pd.read_csv('cleaned_data/stations_2016.csv')
stations_2017 = pd.read_csv('cleaned_data/stations_2017.csv')

In [3]:
stations_2011.head(2)

,station,region,lat,lon
0,5th & F St NW,"Washington, DC",38.897222,-77.019347
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200


In [4]:
df_2011 = pd.read_csv('cleaned_data/df_2011.csv')
df_2012 = pd.read_csv('cleaned_data/df_2012.csv')
df_2013 = pd.read_csv('cleaned_data/df_2013.csv')
df_2014 = pd.read_csv('cleaned_data/df_2014.csv')
df_2015 = pd.read_csv('cleaned_data/df_2015.csv')
df_2016 = pd.read_csv('cleaned_data/df_2016.csv')
df_2017 = pd.read_csv('cleaned_data/df_2017.csv')

In [5]:
df_2011.head(2)

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,registered,casual
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,1,0
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,0,1


<h3> Create dataframes for each year, with full info on each station </h3>

This is the automated version of what I did in plot_geopandas_part_1.ipynb

In [6]:
df_2011.head()

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,registered,casual
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,1,0
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member,1,0
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member,1,0
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual,0,1


In [7]:
start = df_2011.groupby('start_station').agg({'start_station':'count', 'registered': 'sum', 'casual': 'sum' }).rename(columns = {'start_station':'start_total', 'registered': 'start_registered', 'casual':'start_casual'})
end = df_2011.groupby('end_station').agg({'end_station':'count', 'registered': 'sum', 'casual': 'sum' }).rename(columns = {'end_station':'end_total', 'registered': 'end_registered', 'casual':'end_registered'})

In [8]:
start.head()
end.head()

,end_total,end_registered,end_registered
end_station,,,
10th & Monroe St NE,2056,1747,309
10th & U St NW,17247,15345,1902
10th St & Constitution Ave NW,19220,10208,9011
11th & F St NW,454,436,18
11th & H St NE,276,263,13


In [9]:
stations_2011.head(2)

,station,region,lat,lon
0,5th & F St NW,"Washington, DC",38.897222,-77.019347
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200


In [10]:
element = stations_2011.merge(start, how = 'left', left_on = 'station', right_on = 'start_station')
element = element.merge(end, how = 'left', left_on = 'station', right_on = 'end_station')

In [11]:
element.head()

,station,region,lat,lon,start_total,start_registered,start_casual,end_total,end_registered,end_registered
0,5th & F St NW,"Washington, DC",38.897222,-77.019347,14942,11998,2944,16137,12824,3313
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200,17507,15921,1586,12877,11500,1377
2,Georgia & New Hampshire Ave NW,"Washington, DC",38.936684,-77.024181,10045,9193,852,7308,6716,592
3,10th & U St NW,"Washington, DC",38.917200,-77.025900,16737,14907,1830,17247,15345,1902
4,Adams Mill & Columbia Rd NW,"Washington, DC",38.922925,-77.042581,29964,26056,3908,26248,22658,3590


In [12]:
df_all_years = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017]
stations_all_years = [stations_2011, stations_2012, stations_2013, stations_2014, stations_2015, stations_2016, stations_2017]



stations_all_years_updated = []

for item, element in zip (df_all_years, stations_all_years):
    start = item.groupby('start_station').agg({'start_station':'count', 'registered': 'sum', 'casual': 'sum' }).rename(columns = {'start_station':'start_total', 'registered': 'start_registered', 'casual':'start_casual'}) # "start" = station name + num users as start station
    start = start.sort_values(by = 'start_total', ascending=False)                                                        # sort by ranking
    start.reset_index(inplace=True)

    end = item.groupby('end_station').agg({'end_station':'count', 'registered': 'sum', 'casual': 'sum' }).rename(columns = {'end_station':'end_total', 'registered': 'end_registered', 'casual':'end_casual'})     # "end" = station name + num users as end station
    end = end.sort_values(by = 'end_total', ascending=False)                                                            # sort by ranking
    end.reset_index(inplace=True)

    element = element.merge(start, how = 'left', left_on = 'station', right_on = 'start_station')                   # merge station_20xx with "start"
    element = element.merge(end, how = 'left', left_on = 'station', right_on = 'end_station')                       # merge station_20xx with "end"


    element.drop(['start_station', 'end_station'], axis = 1, inplace = True)

    stations_all_years_updated.append(element)


In [13]:
# Store elements of "stations_all_years_updated" into variables

stations_full_2011 = stations_all_years_updated[0]
stations_full_2012 = stations_all_years_updated[1]
stations_full_2013 = stations_all_years_updated[2]
stations_full_2014 = stations_all_years_updated[3]
stations_full_2015 = stations_all_years_updated[4]
stations_full_2016 = stations_all_years_updated[5]
stations_full_2017 = stations_all_years_updated[6]


In [14]:
# Now we have station name + latitude + longitude + start_count + end_count
stations_full_2017.head()

,station,region,lat,lon,start_total,start_registered,start_casual,end_total,end_registered,end_casual
0,10th & E St NW,"Washington, DC",38.895914,-77.026064,15410,8357,7053,17504,10166,7338
1,10th & Florida Ave NW,"Washington, DC",38.920387,-77.025672,8990,8156,834,8367,7586,781
2,10th & G St NW,"Washington, DC",38.898243,-77.026235,11665,7956,3709,13353,9738,3615
3,10th & K St NW,"Washington, DC",38.902400,-77.026220,19381,14534,4847,19363,13473,5890
4,10th & Monroe St NE,"Washington, DC",38.932457,-76.993534,3601,2998,603,2858,2293,565


In [15]:
# export as csv files for future use

stations_full_2011.to_csv('cleaned_data/stations_full_2011.csv', index = False)
stations_full_2012.to_csv('cleaned_data/stations_full_2012.csv', index = False)
stations_full_2013.to_csv('cleaned_data/stations_full_2013.csv', index = False)
stations_full_2014.to_csv('cleaned_data/stations_full_2014.csv', index = False)
stations_full_2015.to_csv('cleaned_data/stations_full_2015.csv', index = False)
stations_full_2016.to_csv('cleaned_data/stations_full_2016.csv', index = False)
stations_full_2017.to_csv('cleaned_data/stations_full_2017.csv', index = False)

In [16]:
# For each year, create a dataframe sorted by "start_count" and another dataframe sorted by "end_count"

all_dataframes = [stations_full_2011, stations_full_2012, stations_full_2013, stations_full_2014, stations_full_2015, stations_full_2016, stations_full_2017]

start_stations = []
end_stations = []

for df in all_dataframes:
    start = df[['station','lat','lon', 'start_total', 'start_registered', 'start_casual']].sort_values(by ='start_total', ascending = False)
    end = df[['station', 'lat','lon','end_total', 'end_registered', 'end_casual']].sort_values(by ='end_total', ascending = False)

    start_stations.append(start)
    end_stations.append(end)

start_stations_2011 = start_stations[0]
start_stations_2012 = start_stations[1]
start_stations_2013 = start_stations[2]
start_stations_2014 = start_stations[3]
start_stations_2015 = start_stations[4]
start_stations_2016 = start_stations[5]
start_stations_2017 = start_stations[6]

end_stations_2011 = end_stations[0]
end_stations_2012 = end_stations[1]
end_stations_2013 = end_stations[2]
end_stations_2014 = end_stations[3]
end_stations_2015 = end_stations[4]
end_stations_2016 = end_stations[5]
end_stations_2017 = end_stations[6]

In [17]:
# Create separate dataframes for casual and registered users

stations_registered = []
stations_casual = []

stations_full = [stations_full_2011, stations_full_2012, stations_full_2013, stations_full_2014, stations_full_2015, stations_full_2016, stations_full_2017]

for item in stations_full: 
    reg = item.copy(deep=True)[['station', 'region', 'lat', 'lon', 'start_registered', 'end_registered']]
    cas = item.copy(deep = True)[['station', 'region', 'lat', 'lon', 'start_casual', 'end_casual']]
    stations_registered.append(reg)
    stations_casual.append(cas)

stations_reg_2011 = stations_registered[0]
stations_reg_2012 = stations_registered[1]
stations_reg_2013 = stations_registered[2]
stations_reg_2014 = stations_registered[3]
stations_reg_2015 = stations_registered[4]
stations_reg_2016 = stations_registered[5]
stations_reg_2017 = stations_registered[6]

stations_cas_2011 = stations_casual[0]
stations_cas_2012 = stations_casual[1]
stations_cas_2013 = stations_casual[2]
stations_cas_2014 = stations_casual[3]
stations_cas_2015 = stations_casual[4]
stations_cas_2016 = stations_casual[5]
stations_cas_2017 = stations_casual[6]


In [18]:
stations_reg_2011.head(2)

,station,region,lat,lon,start_registered,end_registered
0,5th & F St NW,"Washington, DC",38.897222,-77.019347,11998,12824
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200,15921,11500


In [19]:
stations_cas_2011.head(2)

,station,region,lat,lon,start_casual,end_casual
0,5th & F St NW,"Washington, DC",38.897222,-77.019347,2944,3313
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200,1586,1377


In [20]:
# ranked start stations for registered users

start_reg_2011 = stations_reg_2011.drop('end_registered', axis = 1).sort_values(by = 'start_registered', ascending = False ).reset_index(drop=True)
start_reg_2012 = stations_reg_2012.drop('end_registered', axis = 1).sort_values(by = 'start_registered', ascending = False ).reset_index(drop=True)
start_reg_2013 = stations_reg_2013.drop('end_registered', axis = 1).sort_values(by = 'start_registered', ascending = False ).reset_index(drop=True)
start_reg_2014 = stations_reg_2014.drop('end_registered', axis = 1).sort_values(by = 'start_registered', ascending = False ).reset_index(drop=True)
start_reg_2015 = stations_reg_2015.drop('end_registered', axis = 1).sort_values(by = 'start_registered', ascending = False ).reset_index(drop=True)
start_reg_2016 = stations_reg_2016.drop('end_registered', axis = 1).sort_values(by = 'start_registered', ascending = False ).reset_index(drop=True)
start_reg_2017 = stations_reg_2017.drop('end_registered', axis = 1).sort_values(by = 'start_registered', ascending = False ).reset_index(drop=True)

# ranked start stations for casual users
start_cas_2011 = stations_cas_2011.drop('end_casual', axis = 1).sort_values(by='start_casual', ascending = False).reset_index(drop=True)
start_cas_2012 = stations_cas_2012.drop('end_casual', axis = 1).sort_values(by='start_casual', ascending = False).reset_index(drop=True)
start_cas_2013 = stations_cas_2013.drop('end_casual', axis = 1).sort_values(by='start_casual', ascending = False).reset_index(drop=True)
start_cas_2014 = stations_cas_2014.drop('end_casual', axis = 1).sort_values(by='start_casual', ascending = False).reset_index(drop=True)
start_cas_2015 = stations_cas_2015.drop('end_casual', axis = 1).sort_values(by='start_casual', ascending = False).reset_index(drop=True)
start_cas_2016 = stations_cas_2016.drop('end_casual', axis = 1).sort_values(by='start_casual', ascending = False).reset_index(drop=True)
start_cas_2017 = stations_cas_2017.drop('end_casual', axis = 1).sort_values(by='start_casual', ascending = False).reset_index(drop=True)

# ranked end stations for registered users

end_reg_2011 = stations_reg_2011.drop('start_registered', axis = 1).sort_values(by = 'end_registered', ascending = False ).reset_index(drop=True)
end_reg_2012 = stations_reg_2012.drop('start_registered', axis = 1).sort_values(by = 'end_registered', ascending = False ).reset_index(drop=True)
end_reg_2013 = stations_reg_2013.drop('start_registered', axis = 1).sort_values(by = 'end_registered', ascending = False ).reset_index(drop=True)
end_reg_2014 = stations_reg_2014.drop('start_registered', axis = 1).sort_values(by = 'end_registered', ascending = False ).reset_index(drop=True)
end_reg_2015 = stations_reg_2015.drop('start_registered', axis = 1).sort_values(by = 'end_registered', ascending = False ).reset_index(drop=True)
end_reg_2016 = stations_reg_2016.drop('start_registered', axis = 1).sort_values(by = 'end_registered', ascending = False ).reset_index(drop=True)
end_reg_2017 = stations_reg_2017.drop('start_registered', axis = 1).sort_values(by = 'end_registered', ascending = False ).reset_index(drop=True)

# ranked end stations for casual users

end_cas_2011 = stations_cas_2011.drop('start_casual', axis = 1).sort_values(by = 'end_casual', ascending = False ).reset_index(drop=True)
end_cas_2012 = stations_cas_2012.drop('start_casual', axis = 1).sort_values(by = 'end_casual', ascending = False ).reset_index(drop=True)
end_cas_2013 = stations_cas_2013.drop('start_casual', axis = 1).sort_values(by = 'end_casual', ascending = False ).reset_index(drop=True)
end_cas_2014 = stations_cas_2014.drop('start_casual', axis = 1).sort_values(by = 'end_casual', ascending = False ).reset_index(drop=True)
end_cas_2015 = stations_cas_2015.drop('start_casual', axis = 1).sort_values(by = 'end_casual', ascending = False ).reset_index(drop=True)
end_cas_2016 = stations_cas_2016.drop('start_casual', axis = 1).sort_values(by = 'end_casual', ascending = False ).reset_index(drop=True)
end_cas_2017 = stations_cas_2017.drop('start_casual', axis = 1).sort_values(by = 'end_casual', ascending = False ).reset_index(drop=True)

In [21]:
#start_reg_2011.head(2)
#start_cas_2011.head(2)
end_reg_2011.head(5)
end_cas_2011.head(5)

,station,region,lat,lon,end_casual
0,USDA / 12th & Independence Ave SW,0.0,38.887841,-77.029994,13758
1,Massachusetts Ave & Dupont Circle NW,"Washington, DC",38.910100,-77.044400,11339
2,19th St & Constitution Ave NW,"Washington, DC",38.892300,-77.043600,9759
3,10th St & Constitution Ave NW,"Washington, DC",38.893028,-77.026013,9011
4,C & O Canal & Wisconsin Ave NW,"Washington, DC",38.903750,-77.062690,8008


In [22]:
"""
# Let's select the top 10 start and end stations for each year

# Choose top 10
num = 20

top_start_2011 = start_stations_2011.copy(deep = True).iloc[:num,:]
top_start_2012 = start_stations_2012.copy(deep = True).iloc[:num,:]
top_start_2013 = start_stations_2013.copy(deep = True).iloc[:num,:]
top_start_2014 = start_stations_2014.copy(deep = True).iloc[:num,:]
top_start_2015 = start_stations_2015.copy(deep = True).iloc[:num,:]
top_start_2016 = start_stations_2016.copy(deep = True).iloc[:num,:]
top_start_2017 = start_stations_2017.copy(deep = True).iloc[:num,:]

top_end_2011 = end_stations_2011.copy(deep=True).iloc[:num,:]
top_end_2012 = end_stations_2011.copy(deep=True).iloc[:num,:]
top_end_2013 = end_stations_2011.copy(deep=True).iloc[:num,:]
top_end_2014 = end_stations_2011.copy(deep=True).iloc[:num,:]
top_end_2015 = end_stations_2011.copy(deep=True).iloc[:num,:]
top_end_2016 = end_stations_2011.copy(deep=True).iloc[:num,:]
top_end_2017 = end_stations_2011.copy(deep=True).iloc[:num,:]
"""

"\n# Let's select the top 10 start and end stations for each year\n\n# Choose top 10\nnum = 20\n\ntop_start_2011 = start_stations_2011.copy(deep = True).iloc[:num,:]\ntop_start_2012 = start_stations_2012.copy(deep = True).iloc[:num,:]\ntop_start_2013 = start_stations_2013.copy(deep = True).iloc[:num,:]\ntop_start_2014 = start_stations_2014.copy(deep = True).iloc[:num,:]\ntop_start_2015 = start_stations_2015.copy(deep = True).iloc[:num,:]\ntop_start_2016 = start_stations_2016.copy(deep = True).iloc[:num,:]\ntop_start_2017 = start_stations_2017.copy(deep = True).iloc[:num,:]\n\ntop_end_2011 = end_stations_2011.copy(deep=True).iloc[:num,:]\ntop_end_2012 = end_stations_2011.copy(deep=True).iloc[:num,:]\ntop_end_2013 = end_stations_2011.copy(deep=True).iloc[:num,:]\ntop_end_2014 = end_stations_2011.copy(deep=True).iloc[:num,:]\ntop_end_2015 = end_stations_2011.copy(deep=True).iloc[:num,:]\ntop_end_2016 = end_stations_2011.copy(deep=True).iloc[:num,:]\ntop_end_2017 = end_stations_2011.copy(d

In [23]:
# Colour coordinate dataframes by number of users

start_reg_2017.head(10)
start_cas_2017.head(10)

,station,region,lat,lon,start_casual
0,Lincoln Memorial,"Washington, DC",38.888255,-77.049436,52320
1,Jefferson Dr & 14th St SW,"Washington, DC",38.888553,-77.032427,46621
2,Henry Bacon Dr & Lincoln Memorial Circle NW,"Washington, DC",38.890539,-77.049383,34143
3,Smithsonian-National Mall / Jefferson Dr & 12t...,"Washington, DC",38.888774,-77.028694,33351
4,4th St & Madison Dr NW,"Washington, DC",38.890496,-77.017246,28145
5,Jefferson Memorial,"Washington, DC",38.879819,-77.037413,27830
6,17th St & Independence Ave SW,"Washington, DC",38.888097,-77.038325,24663
7,Constitution Ave & 2nd St NW/DOL,"Washington, DC",38.892275,-77.013917,17245
8,4th & C St SW,"Washington, DC",38.886372,-77.017701,16877
9,Ohio Dr & West Basin Dr SW / MLK & FDR Memorials,"Washington, DC",38.884120,-77.046570,14780


In [24]:
all_start_reg = [start_reg_2011, start_reg_2012, start_reg_2013, start_reg_2014, start_reg_2015, start_reg_2016, start_reg_2017]
all_end_reg = [end_reg_2011, end_reg_2012, end_reg_2013, end_reg_2014, end_reg_2015, end_reg_2016, end_reg_2017]
all_start_cas = [start_cas_2011, start_cas_2012, start_cas_2013, start_cas_2014, start_cas_2015, start_cas_2016, start_cas_2017]
all_end_cas = [end_cas_2011, end_cas_2012, end_cas_2013, end_cas_2014, end_cas_2015, end_cas_2016, end_cas_2017 ]

num_years = 7

for i in range(num_years):
    all_start_reg[i]['marker_colour'] = all_start_reg[i]['start_registered'].apply(lambda x: '#D2042D' if x >= 50000  # red
                                                                            else '#FFEA00' if x >= 40000 # yellow
                                                                            else '#00FF00' if x >= 30000 # lime
                                                                            else '#0000FF' if x >= 20000  # blue
                                                                            else '#808080')                 # grey
    all_end_reg[i]['marker_colour'] = all_end_reg[i]['end_registered'].apply(lambda x: '#D2042D' if x >= 50000  
                                                                            else '#FFEA00' if x >= 40000 
                                                                            else '#00FF00' if x >= 30000 
                                                                            else '#0000FF' if x >= 20000  
                                                                            else '#808080')

    all_start_cas[i]['marker_colour'] = all_start_cas[i]['start_casual'].apply(lambda x: '#D2042D' if x >= 50000  
                                                                            else '#FFEA00' if x >= 40000 
                                                                            else '#00FF00' if x >= 30000 
                                                                            else '#0000FF' if x >= 20000  
                                                                            else '#808080')

    all_end_cas[i]['marker_colour'] = all_end_cas[i]['end_casual'].apply(lambda x: '#D2042D' if x >= 50000  
                                                                            else '#FFEA00' if x >= 40000 
                                                                            else '#00FF00' if x >= 30000 
                                                                            else '#0000FF' if x >= 20000  
                                                                            else '#808080')

In [25]:
start_reg_2011
end_reg_2011
start_cas_2011
end_cas_2011

,station,region,lat,lon,end_casual,marker_colour
0,USDA / 12th & Independence Ave SW,0.0,38.887841,-77.029994,13758,#808080
1,Massachusetts Ave & Dupont Circle NW,"Washington, DC",38.910100,-77.044400,11339,#808080
2,19th St & Constitution Ave NW,"Washington, DC",38.892300,-77.043600,9759,#808080
3,10th St & Constitution Ave NW,"Washington, DC",38.893028,-77.026013,9011,#808080
4,C & O Canal & Wisconsin Ave NW,"Washington, DC",38.903750,-77.062690,8008,#808080
...,...,...,...,...,...,...
139,Nannie Helen Burroughs & Minnesota Ave NE,"Washington, DC",38.901385,-76.941877,0,#808080
140,Benning Rd & East Capitol St NE / Benning Rd M...,"Washington, DC",38.889935,-76.937230,0,#808080
141,Anacostia Ave & Benning Rd NE / River Terrace,"Washington, DC",38.896544,-76.960120,0,#808080
142,15th St & Massachusetts Ave SE,"Washington, DC",38.887320,-76.983569,0,#808080


<h3> Washington DC zoning map </h3>

In [26]:
zoning = gpd.read_file("shp_files/Zoning_Regulations_of_2016.geojson")

In [27]:
zoning.head(2)

,ZONING,ZONING_WEB_URL,ZONING_CHANGE_NARRATIVE,ZONING_STATUS,ZONING_LABEL,ZONE_DESCRIPTION,ZONE_DISTRICT,IZ_DESIGNATION,ZR58,ZR16,GLOBALID,OBJECTID,SHAPEAREA,SHAPELEN,geometry
0,MU-7B,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Fixed map error of SSL 4507 936, 07/14/11, Mis...",Final,MU-7B,Permits medium density mixed-use development,Mixed-Use Zone,None,C-3-A,MU-7B,{CA941084-283B-4ADB-BDB4-2F5F4D5F8C33},515010,0,0,"POLYGON ((-76.98059 38.90251, -76.98070 38.902..."
1,MU-4,https://handbook.dcoz.dc.gov/zones/mixed-use/M...,"Official Digital Zoning Map of July 1, 2010",Final,MU-4,Permits moderate density mixed use development,Mixed-Use Zone,None,C-2-A,MU-4,{EE555AF3-9EAA-4FBB-997A-CA12025CBCD6},515011,0,0,"POLYGON ((-76.99314 38.90401, -76.99304 38.903..."


In [28]:
zoning.drop(['ZONING_WEB_URL', 'ZONING_CHANGE_NARRATIVE', 'ZONING_STATUS', 
            'ZONING_LABEL', 'ZONE_DESCRIPTION', 'IZ_DESIGNATION','OBJECTID', 'SHAPEAREA'], 
            axis = 1, inplace = True)

In [29]:
zoning['ZONE_DISTRICT'].unique()

array(['Mixed-Use Zone', 'Residential Apartment Zone', 'Downtown Zone',
       'Production, Distribution, and Repair Zone', 'Residential Zone',
       'Unzoned', 'Special Purpose Zone', 'Neighborhood Mixed-Use Zone',
       'Residential Flat Zone'], dtype=object)

In [30]:
# Create new column, "zoning_colour", using a dictionary. 

# Overview of colour assignment
"""
'Mixed-Use Zone', 'Neighborhood Mixed-Use Zone' –> 0
'Residential Apartment Zone', 'Residential Zone', 'Residential Flat Zone' –> 1
'Downtown Zone' –> 4
'Production, Distribution, and Repair Zone' –> 3
'Unzoned' –> 2
'Special Purpose Zone' –> 5
"""

zoning_colour_dict = {'Mixed-Use Zone': 0,
                    'Neighborhood Mixed-Use Zone': 0,
                    'Residential Apartment Zone': 1,
                    'Residential Zone': 1,
                    'Residential Flat Zone': 1,
                    'Downtown Zone': 4,
                    'Production, Distribution, and Repair Zone': 3,
                    'Unzoned': 2,
                    'Special Purpose Zone': 5}

zoning['zone_district_colour'] = zoning['ZONE_DISTRICT'].map(zoning_colour_dict)

In [31]:
zoning.head(2)

,ZONING,ZONE_DISTRICT,ZR58,ZR16,GLOBALID,SHAPELEN,geometry,zone_district_colour
0,MU-7B,Mixed-Use Zone,C-3-A,MU-7B,{CA941084-283B-4ADB-BDB4-2F5F4D5F8C33},0,"POLYGON ((-76.98059 38.90251, -76.98070 38.902...",0
1,MU-4,Mixed-Use Zone,C-2-A,MU-4,{EE555AF3-9EAA-4FBB-997A-CA12025CBCD6},0,"POLYGON ((-76.99314 38.90401, -76.99304 38.903...",0


In [32]:
# Create pure dataframe (without geodata)
zoning_just_df = zoning[['ZONING', 'zone_district_colour']]

In [33]:
start_reg_2011
end_reg_2011
start_cas_2011
end_cas_2011

,station,region,lat,lon,end_casual,marker_colour
0,USDA / 12th & Independence Ave SW,0.0,38.887841,-77.029994,13758,#808080
1,Massachusetts Ave & Dupont Circle NW,"Washington, DC",38.910100,-77.044400,11339,#808080
2,19th St & Constitution Ave NW,"Washington, DC",38.892300,-77.043600,9759,#808080
3,10th St & Constitution Ave NW,"Washington, DC",38.893028,-77.026013,9011,#808080
4,C & O Canal & Wisconsin Ave NW,"Washington, DC",38.903750,-77.062690,8008,#808080
...,...,...,...,...,...,...
139,Nannie Helen Burroughs & Minnesota Ave NE,"Washington, DC",38.901385,-76.941877,0,#808080
140,Benning Rd & East Capitol St NE / Benning Rd M...,"Washington, DC",38.889935,-76.937230,0,#808080
141,Anacostia Ave & Benning Rd NE / River Terrace,"Washington, DC",38.896544,-76.960120,0,#808080
142,15th St & Massachusetts Ave SE,"Washington, DC",38.887320,-76.983569,0,#808080


In [34]:
# TOP 20 START AND END STATIONS FOR REGISTERED AND CASUAL USERS
f = folium.Figure(width=1100, height=600)
m = folium.Map(control_scale=True, location=[38.900497,-77.007507],pointer_events=True, zoom_start = 13)

N = 20

# List of top N start stations for registered and casual users

start_reg_all_years = [start_reg_2011.loc[:N,:], start_reg_2012.loc[:N,:], start_reg_2013.loc[:N,:], start_reg_2014.loc[:N,:], start_reg_2015.loc[:N,:], start_reg_2016.loc[:N,:], start_reg_2017.loc[:N,:] ]
start_cas_all_years = [start_cas_2011.loc[:N, :], start_cas_2012.loc[:N, :], start_cas_2013.loc[:N, :], start_cas_2014.loc[:N, :], start_cas_2015.loc[:N, :], start_cas_2016.loc[:N, :], start_cas_2017.loc[:N, :] ]

# List of top N end stations for registered and casual users

end_reg_all_years = [end_reg_2011.loc[:N, :], end_reg_2012.loc[:N, :], end_reg_2013.loc[:N, :], end_reg_2014.loc[:N, :], end_reg_2015.loc[:N, :], end_reg_2016.loc[:N, :], end_reg_2017.loc[:N, :] ]
end_cas_all_years = [end_cas_2011.loc[:N, :], end_cas_2012.loc[:N, :], end_cas_2013.loc[:N, :], end_cas_2014.loc[:N, :], end_cas_2015.loc[:N, :], end_cas_2016.loc[:N, :], end_cas_2017.loc[:N, :] ]


# List of all years
years_list = ['2011', '2012', '2013','2014', '2015', '2016', '2017' ]

# Create a list of FeatureGroup instances for all years
feature_groups_start_reg = [folium.FeatureGroup(name = f"{year} top 20 start stations (registered)", show = False) for year in years_list]
feature_groups_start_cas = [folium.FeatureGroup(name = f"{year} top 20 start stations (casual)", show = False) for year in years_list]
feature_groups_end_reg = [folium.FeatureGroup(name = f"{year} top 20 end stations (registered)", show = False) for year in years_list]
feature_groups_end_cas = [folium.FeatureGroup(name = f"{year} top 20 end stations (casual)", show = False) for year in years_list]

# Add child for each FeatureGroup instance
for fg in feature_groups_start_reg:
    m.add_child(fg)

for fg in feature_groups_start_cas:
    m.add_child(fg)

for fg in feature_groups_end_reg:
    m.add_child(fg)

for fg in feature_groups_end_cas:
    m.add_child(fg)

# Add circlemarkers to each FeatureGroup instance
for year in range (len(start_reg_all_years)):

    for index, row in start_reg_all_years[year].iterrows():
        
        label = f"Station:{row['station']}"
        label = folium.Tooltip(label)
        folium.CircleMarker([row['lat'], row['lon']], radius = 10, popup = label, 
                            color = row['marker_colour'], fill = True, 
                            fill_color = row['marker_colour']).add_to(feature_groups_start_reg[year])

for year in range (len(start_cas_all_years)):

    for index, row in start_cas_all_years[year].iterrows():
        
        label = f"Station:{row['station']}"
        label = folium.Tooltip(label)
        folium.CircleMarker([row['lat'], row['lon']], radius = 10, popup = label, 
                            color = row['marker_colour'], fill = True, 
                            fill_color = row['marker_colour']).add_to(feature_groups_start_cas[year])

for year in range (len(end_reg_all_years)):

    for index, row in end_reg_all_years[year].iterrows():
        
        label = f"Station:{row['station']}"
        label = folium.Tooltip(label)
        folium.CircleMarker([row['lat'], row['lon']], radius = 10, popup = label, 
                            color = row['marker_colour'], fill = True, 
                            fill_color = row['marker_colour']).add_to(feature_groups_end_reg[year])

for year in range (len(end_cas_all_years)):

    for index, row in end_cas_all_years[year].iterrows():
        
        label = f"Station:{row['station']}"
        label = folium.Tooltip(label)
        folium.CircleMarker([row['lat'], row['lon']], radius = 10, popup = label, 
                            color = row['marker_colour'], fill = True, 
                            fill_color = row['marker_colour']).add_to(feature_groups_end_cas[year])


        
# Add zoning map layer
folium.Choropleth(geo_data= zoning, data = zoning_just_df, columns = ['ZONING', 'zone_district_colour'],
                    key_on='feature.properties.ZONING', fill_opacity=0.3,
                    fill_color='Set1', legend_name='mixed-use  |  residential    |    unzoned    |    production     |     downtown     |     special').add_to(m)

# Add every FeatureGroup instance to base map "m"
for fg in feature_groups_start_reg:
    fg.add_to(m)

for fg in feature_groups_start_cas:
    fg.add_to(m)

for fg in feature_groups_end_reg:
    fg.add_to(m)

for fg in feature_groups_end_cas:
    fg.add_to(m)

# Enable LayerControl
folium.LayerControl().add_to(m)    
f.add_child(m)
f

outfp = "DC_bike_map.html"
f.save(outfp)

In [35]:
# Now automate heatmap generation for all years

m=folium.Map(location=[38.900497,-77.007507],pointer_events=True)

years_list = ['2011', '2012', '2013','2014', '2015', '2016', '2017' ]
heatmap_figs = [folium.FeatureGroup(name=f"{year} heat map", show = False) for year in years_list]

for heatmap_fig in heatmap_figs:
    m.add_child(heatmap_fig)

all_start_stations = [start_stations_2011, start_stations_2012, start_stations_2013, 
                    start_stations_2014, start_stations_2015, start_stations_2016, 
                    start_stations_2017]

for i, start_station in enumerate(all_start_stations):
    heat_data = [[row['lat'],row['lon']] for index, row in start_station.iterrows()]
    HeatMap(heat_data).add_to(heatmap_figs[i])

folium.LayerControl().add_to(m)
m

outfp = "DC_bike_heat_map.html"
m.save(outfp)